In [2]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.Collecting statsmodels
  Using cached statsmodels-0.14.2-cp312-cp312-win_amd64.whl.metadata (9.5 kB)
   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
    --------------------------------------- 0.2/9.8 MB ? eta -:--:--
    --------------------------------------- 0.2/9.8 MB ? eta -:--:--
    --------------------------------------- 0.2/9.8 MB ? eta -:--:--
    --------------------------------------- 0.2/9.8 MB ? eta -:--:--
    --------------------------------------- 0.2/9.8 MB 1.1 MB/s eta 0:00:09
    --------------------------------------- 0.2/9.8 MB 1.1 MB/s eta 0:00:09
    --------------------------------------- 0.2/9.8 MB 1.1 MB/s eta 0:00:09
    --------------------------------------- 0.2/9.8 MB 1.1 MB/s eta 0:00:09
    --------------------------------------- 0.2/9.8 MB 1.1 MB/s eta 0:00:09
    --------------------------------------- 0.2/9.8 MB 1.1 MB/s eta 0:00:09
    -----------------------------


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Importing Libaries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statsmodels.api as sm

## Understanding the data

In [ ]:
df = pd.read_csv("C:/Users/2024/llm/LLM-Hackathon/src/data/sales.csv")
df.head()

In [15]:
df['Item Name'] = df['Item Name'].str.split().str[:3].str.join(' ')
df['Item Name'] = df['Item Name'].str.replace(r'\b\d+\s*G\b', '', regex=True)

# Trim whitespace from the 'Item Name' column
df['Item Name'] = df['Item Name'].str.strip()
df.to_csv('Updated_sales.csv', index=False)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1399 entries, 0 to 1398
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Period                 1399 non-null   object
 1   City                   1399 non-null   object
 2   Channel                1399 non-null   object
 3   Category               1399 non-null   object
 4   Segment                1399 non-null   object
 5   Manufacturer           1399 non-null   object
 6   Brand                  1399 non-null   object
 7   Item Name              1399 non-null   object
 8   Pack_Size              1399 non-null   object
 9   Packaging              1399 non-null   object
 10  Unit_Price             1399 non-null   object
 11  Sales_Volume(KG_LTRS)  1399 non-null   object
 12  Sales_Value            1399 non-null   object
dtypes: object(13)
memory usage: 142.2+ KB


In [9]:
df.columns

Index(['Period', 'City', 'Channel', 'Category', 'Segment', 'Manufacturer',
       'Brand', 'Item Name', 'Pack_Size', 'Packaging', 'Unit_Price',
       'Sales_Volume(KG_LTRS)', 'Sales_Value'],
      dtype='object')

In [10]:
df.isnull().sum()

Period                   0
City                     0
Channel                  0
Category                 0
Segment                  0
Manufacturer             0
Brand                    0
Item Name                0
Pack_Size                0
Packaging                0
Unit_Price               0
Sales_Volume(KG_LTRS)    0
Sales_Value              0
dtype: int64

In [13]:
df.describe().T

,count,unique,top,freq
Period,1399,24,Aug-21,68
City,1399,2,Abidjan,723
Channel,1399,3,Groceries,602
Category,1399,1,PASTA,1399
Segment,1399,1,DRY PASTA,1399
Manufacturer,1399,12,CAPRA,1163
Brand,1399,16,MAMAN,974
Item Name,1399,28,MAMAN SPAGHETTI,192
Pack_Size,1399,6,200G,1009
Packaging,1399,2,SACHET,1294


In [17]:
df.head()

,Period,City,Channel,Category,Segment,Manufacturer,Brand,Item Name,Pack_Size,Packaging,Unit_Price,Sales_Volume(KG_LTRS),Sales_Value
0,Jan-21,Abidjan,Boutique,PASTA,DRY PASTA,CAPRA,ALYSSA,ALYSSA SPAGHETTI,200G,SACHET,89.06,"66,795.7","21,286,480.6"
1,Jan-21,Abidjan,Boutique,PASTA,DRY PASTA,CAPRA,MAMAN,MAMAN SUPERIOR QUALITY,200G,SACHET,99.75,"72,729.2","26,580,841.7"
2,Jan-21,Abidjan,Boutique,PASTA,DRY PASTA,CAPRA,MAMAN,MAMAN VERMICELLI,200G,SACHET,99.75,"56,039.7","20,481,238.0"
3,Jan-21,Abidjan,Boutique,PASTA,DRY PASTA,CAPRA,MAMAN,MAMAN 1.1 SPAGHETTI,200G,SACHET,99.75,"18,220.5","6,659,200.3"
4,Jan-21,Abidjan,Boutique,PASTA,DRY PASTA,CAPRA,MAMAN,MAMAN 1.5 SPAGHETTI,200G,SACHET,99.75,"376,047.6","136,381,702.0"


In [ ]:
total_sales_value = df['Sales_Value'].sum()

# Group by Manufacturer and sum the Sales_Value
manufacturer_sales = df.groupby('Manufacturer')['Sales_Value'].sum().reset_index()

# Calculate market share
manufacturer_sales['Market Share'] = (manufacturer_sales['Sales_Value'] / total_sales_value) * 100

# Display the table
print(manufacturer_sales)

## Visual Understanding

In [8]:
sales_by_brand = df.groupby('Brand')['Sales_Value'].sum().reset_index()
sales_by_brand.columns = ['Brand', 'Total Sales']

fig = px.bar(sales_by_brand, x='Brand', y='Total Sales', title='Total Sales by Brand')
fig.show()

In [9]:
df['Period'] = pd.to_datetime(df['Period'])

# Aggregating the data
aggregates = {
    'City': df.groupby('City')[['Sales_Volume(KG_LTRS)', 'Sales_Value']].sum().reset_index(),
    'Channel': df.groupby('Channel')[['Sales_Volume(KG_LTRS)', 'Sales_Value']].sum().reset_index(),
    'Category': df.groupby('Category')[['Sales_Volume(KG_LTRS)', 'Sales_Value']].sum().reset_index(),
    'Segment': df.groupby('Segment')[['Sales_Volume(KG_LTRS)', 'Sales_Value']].sum().reset_index(),
    'Manufacturer': df.groupby('Manufacturer')[['Sales_Volume(KG_LTRS)', 'Sales_Value']].sum().reset_index(),
    'Brand': df.groupby('Brand')[['Sales_Volume(KG_LTRS)', 'Sales_Value']].sum().reset_index(),
    'Item Name': df.groupby('Item Name')[['Sales_Volume(KG_LTRS)', 'Sales_Value']].sum().reset_index()
}

# Plotting function
def plot_top_performers(df, category):
    fig = px.bar(df.sort_values(by='Sales_Value', ascending=False).head(10), 
                 x=category, 
                 y='Sales_Value', 
                 title=f'Top 10 {category}s by Sales Value',
                 labels={category: category, 'Sales_Value': 'Sales Value'},
                 text='Sales_Value')
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
    fig.show()

# Plotting top performers
for category, df in aggregates.items():
    plot_top_performers(df, category)

In [19]:
df['Period'] = pd.to_datetime(df['Period'])

# Function to plot sales trends over time for a given category
def plot_sales_trend(data, category):
    aggregated_data = data.groupby([category, 'Period'])[['Sales_Volume(KG_LTRS)', 'Sales_Value']].sum().reset_index()
    
    fig = px.line(aggregated_data, 
                  x='Period', 
                  y='Sales_Value', 
                  color=category, 
                  title=f'Sales Trend Over Time by {category}',
                  labels={'Sales_Value': 'Sales Value', 'Period': 'Period'},
                  line_group=category,
                  hover_name=category)
    
    fig.update_layout(xaxis_title='Period', yaxis_title='Sales Value')
    fig.show()

# Plot sales trends for different categories
categories = ['City', 'Channel', 'Category', 'Segment', 'Manufacturer', 'Brand']

for category in categories:
    plot_sales_trend(df, category)

In [21]:
df['Period'] = pd.to_datetime(df['Period'])

# Extract month and year from the 'Period' column
df['Month'] = df['Period'].dt.month
df['Year'] = df['Period'].dt.year

monthly_sales_city = df.groupby(['Month', 'City'])[['Sales_Volume(KG_LTRS)', 'Sales_Value']].sum().reset_index()

# Plot seasonal patterns for each city
def plot_seasonal_patterns_by_category(monthly_data, category):
    fig = px.line(monthly_data, 
                  x='Month', 
                  y='Sales_Value', 
                  color=category,
                  title=f'Seasonal Patterns in Sales by {category}',
                  labels={'Sales_Value': 'Sales Value', 'Month': 'Month'},
                  markers=True)
    fig.update_layout(xaxis=dict(tickmode='array', tickvals=list(range(1, 13)), ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']))
    fig.show()

# Plot seasonal patterns for cities
plot_seasonal_patterns_by_category(monthly_sales_city, 'City')

In [3]:
# Convert 'Period' column to datetime
df['Period'] = pd.to_datetime(df['Period'])

# Scatter plot for Unit Price vs Sales Volume
fig1 = px.scatter(df, x='Unit_Price', y='Sales_Volume(KG_LTRS)',
                  title='Unit Price vs Sales Volume',
                  labels={'Unit_Price': 'Unit Price', 'Sales_Volume(KG_LTRS)': 'Sales Volume (KG/LTRS)'},
                  trendline='ols')
fig1.show()

# Scatter plot for Unit Price vs Sales Value
fig2 = px.scatter(df, x='Unit_Price', y='Sales_Value',
                  title='Unit Price vs Sales Value',
                  labels={'Unit_Price': 'Unit Price', 'Sales_Value': 'Sales Value'},
                  trendline='ols')
fig2.show()

# Regression analysis for Unit Price vs Sales Volume
X_volume = sm.add_constant(df['Unit_Price'])
model_volume = sm.OLS(df['Sales_Volume(KG_LTRS)'], X_volume).fit()
print(model_volume.summary())

# Regression analysis for Unit Price vs Sales Value
X_value = sm.add_constant(df['Unit_Price'])
model_value = sm.OLS(df['Sales_Value'], X_value).fit()
print(model_value.summary())

                              OLS Regression Results                             
Dep. Variable:     Sales_Volume(KG_LTRS)   R-squared:                       0.027
Model:                               OLS   Adj. R-squared:                  0.026
Method:                    Least Squares   F-statistic:                     38.53
Date:                   Wed, 03 Jul 2024   Prob (F-statistic):           7.10e-10
Time:                           11:03:09   Log-Likelihood:                -17017.
No. Observations:                   1399   AIC:                         3.404e+04
Df Residuals:                       1397   BIC:                         3.405e+04
Df Model:                              1                                         
Covariance Type:               nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.094e+04 